# Title

#### Explain

Importamos librerías

In [1]:
from pymongo import MongoClient
import pandas as pd
from src.gameslib import (geocode,gMapsLink,toCartoPoint,toGeoJSON)
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
import re

Indicamos la base de datos de MongoDB (en carpeta /output, unzip y cargar en servidor local mongodb)

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

Tomamos una selección de campos para la nueva colección.

In [3]:

select = {"name":1,"category_code":1,"number_of_employees":1,"tag_list":1,"offices":1, "_id":0}

companies2 = list(db["companies"].find({},select))


In [4]:

df = pd.DataFrame(companies2)
df.head()

,name,category_code,number_of_employees,tag_list,offices
0,AdventNet,enterprise,600.0,,"[{'description': 'Headquarters', 'address1': '..."
1,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...","[{'description': '', 'address1': '710 - 2nd Av..."
2,Zoho,software,1600.0,"zoho, officesuite, spreadsheet, writer, projec...","[{'description': 'Headquarters', 'address1': '..."
3,Digg,news,60.0,"community, social, news, bookmark, digg, techn...","[{'description': None, 'address1': '135 Missis..."
4,Omnidrive,network_hosting,NaN,"storage, sharing, edit, online","[{'description': '', 'address1': 'Suite 200', ..."


Como vemos arriba, hay que *desmembrar* la columna de offices para sacar de la lista de diccionarios las coordenadas de las distintas ubicaciones

In [5]:
#explode and expand
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)

Drop NaN in coords

In [6]:
cleanData.dropna(subset=['latitude','longitude'],inplace=True)
cleanData.reset_index(inplace=True,drop=True)
cleanData.drop(columns=["offices"],inplace=True)

cleanData.fillna(value={'number_of_employees':5.0},inplace=True)
cleanData.head()

,name,category_code,number_of_employees,tag_list,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,enterprise,600.0,,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,Wetpaint,web,47.0,"wiki, seattle, elowitz, media-industry, media-...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,Zoho,software,1600.0,"zoho, officesuite, spreadsheet, writer, projec...",Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
4,Digg,news,60.0,"community, social, news, bookmark, digg, techn...",None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523


Con la longitud y latitud limpias y sin nulos, creamos una columna tipo POINT para cartoframe y otra tipo geoJSON para incorporar a MongoDB

In [7]:
toCartoPoint(-0.116537,51.517742)

'POINT (-0.116537 51.517742)'

In [8]:
toGeoJSON(-0.116537,51.517742)

{'type': 'Point', 'coordinates': [-0.116537, 51.517742]}

In [9]:
cleanData["geometry"] = cleanData[["latitude","longitude"]].apply(lambda x:toCartoPoint(x.longitude,x.latitude), axis=1)
cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:toGeoJSON(x.longitude,x.latitude), axis=1)
cleanData[["latitude","longitude","geometry","location"]].head()

,latitude,longitude,geometry,location
0,37.692934,-121.904945,POINT (-121.904945 37.692934),"{'type': 'Point', 'coordinates': [-121.904945,..."
1,47.603122,-122.333253,POINT (-122.333253 47.603122),"{'type': 'Point', 'coordinates': [-122.333253,..."
2,40.723731,-73.996431,POINT (-73.9964312 40.7237306),"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,37.692934,-121.904945,POINT (-121.904945 37.692934),"{'type': 'Point', 'coordinates': [-121.904945,..."
4,37.764726,-122.394523,POINT (-122.394523 37.764726),"{'type': 'Point', 'coordinates': [-122.394523,..."


Exportamos a json para incluirla como nueva colección en mongodb

In [10]:
cleanData.to_json("./inputs/cleaned_offices.json", orient="records")

# In bash: 
# $ mongoimport --db companies --collection cleancompanies --jsonArray --drop inputs/cleaned_offices.json

# (le estamos indicando que queremos importar nuestro json a la db companies con nombre cleancompanies)


#### Para visualizar los datos mediante mapas en Mongo Compass, iremos a nuestra nueva colección, pestaña Indexes > Create Index > nombre (p.ej. "geoindex") > Seleccionamos columna "location" y '2dsphere' en el desplegable

In [11]:

ciudad = geocode("Köln")
print(ciudad)
#print(gMapsLink(ciudad))

{'type': 'Point', 'coordinates': [6.9665, 50.93545]}


In [12]:

print(gMapsLink(ciudad))

https://www.google.com/maps/@50.93545,6.9665,14z


In [13]:
lat='38.345887'
long='-0.490576'
tipositio='school'
radio='2500'
API_KEY = os.getenv("PLACES_API_TOKEN")
URL = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{long}&type={tipositio}&radius={radio}&key={API_KEY}'


In [14]:
response_data = requests.get(URL)


In [15]:
data = len(list(response_data))
data

254

In [16]:
json_data = json.loads(response_data.text)
lista_coles = json_data['results']
for i in range(len(lista_coles)):
    print(lista_coles[i]['geometry']['location'])

{'lat': 38.3567087, 'lng': -0.5058289}
{'lat': 38.34958, 'lng': -0.4740800000000001}
{'lat': 38.3523635, 'lng': -0.5051052}
{'lat': 38.34316400000001, 'lng': -0.492161}
{'lat': 38.3619991, 'lng': -0.4909873000000001}
{'lat': 38.3534778, 'lng': -0.4856677}
{'lat': 38.3466503, 'lng': -0.5062078999999999}
{'lat': 38.3392515, 'lng': -0.5077184}
{'lat': 38.3553565, 'lng': -0.4739583999999999}
{'lat': 38.3447153, 'lng': -0.4900758}
{'lat': 38.345251, 'lng': -0.491248}
{'lat': 38.3411174, 'lng': -0.4958768}
{'lat': 38.3432417, 'lng': -0.4840492}
{'lat': 38.35493140000001, 'lng': -0.4869952999999999}
{'lat': 38.3400287, 'lng': -0.4991959}
{'lat': 38.35236520000001, 'lng': -0.5032013}
{'lat': 38.3534297, 'lng': -0.4964602}
{'lat': 38.34400730000001, 'lng': -0.4859191}
{'lat': 38.35245219999999, 'lng': -0.4748248999999999}
{'lat': 38.3486923, 'lng': -0.4747926}


In [17]:
# From Response/JSON to dict:
# data_dict = json.loads(response_data.text)
# From dict to json:
# y = json.dumps(data_dict)



In [18]:
lat='38.345887'
long='-0.490576'
radio='1000'
placename = 'Starbucks'
URL = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={placename}&inputtype=textquery&fields=formatted_address,name&locationbias=circle:{radio}@{lat},{long}&key={API_KEY}'

In [19]:
response_data = requests.get(URL)


In [20]:
json_data = json.loads(response_data.text)
#starb = json_data['results']
json_data

{'candidates': [{'formatted_address': 'Av. Maisonnave, 53, 03003 Alicante, España',
   'name': 'Starbucks'}],
 'status': 'OK'}